In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from os.path import join
from os import getcwd
from IPython.display import clear_output

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, auc

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.metrics import AUC # Area under the curve, default: ROC
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, TensorBoard
from keras.optimizers.schedules import ExponentialDecay
from keras.initializers import GlorotNormal
import kerastuner as kt

In [4]:
from functions import *

In [5]:
checkpoints_path = getcwd()+'\\checkpoints_A'
tensor_path = getcwd()+'\\tensor_A'

In [6]:
df = pd.read_csv('../../databases/diabetes.csv')
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
outlayers = {
    'BloodPressure': (40, np.Inf),
    'SkinThickness': (0, 80),
    'Insulin': (0, 400),
    'BMI': (0, 50)
}

zeros = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]
x_df = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']]
y_df = df['Outcome']

In [8]:
# Split dataset into 15% test, 85% train 
x_temp, x_test, y_temp, y_test = train_test_split(x_df, y_df, test_size=0.15, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_temp, y_temp, test_size=0.15, random_state=1)

# Regresión Logistica

### Con los datos sin procesar

In [9]:
name = 'basic'
checkdir = join(checkpoints_path,name)
tensordir = join(tensor_path, name)

model_0 = Sequential()
model_0.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_0.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [10]:
model_0_checkpoint_callback = ModelCheckpoint(
    filepath=checkdir,
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

tensor_call = TensorBoard(log_dir=tensordir)

In [11]:
%%time
history = model_0.fit(x_train, y_train, epochs=50, 
            validation_data=(x_valid, y_valid),
            callbacks=[model_0_checkpoint_callback, tensor_call],
            verbose=0)
# Cargo el mejor modelo entrenado
model_0.load_weights(checkdir)
verify_model(model_0, x_train, y_train, x_valid, y_valid)

Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Wall time: 9.39 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.675764,0.628809,0.642487,0.480620,0.766892
1,Validacion,0.750460,0.703125,0.676471,0.547619,0.803571


### Reemplazando datos invalidos

In [12]:
x_train_nonzero, _data = replace_outliers_zeros(x_train, {}, zeros, mean_median=True)
x_test_nonzero, _data = replace_outliers_zeros(x_test, {}, zeros, mean_median=True, data_to_replace=_data)
x_valid_nonzero, _data = replace_outliers_zeros(x_valid, {}, zeros, mean_median=True, data_to_replace=_data)

In [13]:
name = 'valid'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)

model_1 = Sequential()
model_1.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_1.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_1_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [14]:
%%time 
history = model_1.fit(x_train_nonzero, y_train, epochs=50, validation_data=(x_valid_nonzero, y_valid), callbacks=[model_1_checkpoint_callback, tensor_call], verbose=0)
# Cargo el mejor modelo entrenado
model_1.load_weights(checkdir)
verify_model(model_1, x_train_nonzero, y_train, x_valid_nonzero, y_valid)

Wall time: 12.4 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.705854,0.686981,0.652850,0.527197,0.787302
1,Validacion,0.735983,0.703125,0.647059,0.536585,0.789474


Se puede observar como en ambos casos se obtuvo un mejor resultado en validacion que en train, lo cual es un comportamiento que se puede considerar poco esperado

### Reemplazando outlayers

In [15]:
x_train_clean, _data = replace_outliers_zeros(x_train, outlayers, zeros, mean_median=True)
x_test_clean, _data = replace_outliers_zeros(x_test, outlayers, zeros, mean_median=True, data_to_replace=_data)
x_valid_clean, _data = replace_outliers_zeros(x_valid, outlayers, zeros, mean_median=True, data_to_replace=_data)

In [16]:
name = 'sin_outlayers'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)

model_2 = Sequential()
model_2.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_clean.shape[1],)))
model_2.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_2_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [17]:
%%time 
history = model_2.fit(x_train_clean, y_train, epochs=50, validation_data=(x_valid_clean, y_valid), callbacks=[model_2_checkpoint_callback, tensor_call], verbose=0)
# Cargo el mejor modelo entrenado
model_2.load_weights(checkdir)
verify_model(model_2, x_train_clean, y_train, x_valid_clean, y_valid)

Wall time: 13.8 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.709687,0.728532,0.632124,0.554545,0.787425
1,Validacion,0.732537,0.734375,0.647059,0.564103,0.796610


### Normalizando los datos de entrada

In [18]:
x_train_norm, _norm_dict = normalize(x_train_clean, None)
x_valid_norm, _norm_dict = normalize(x_valid_clean, _norm_dict)
x_test_norm, _norm_dict = normalize(x_test_clean, _norm_dict)

In [19]:
name = 'normalizado'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)

model_3 = Sequential()
model_3.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
model_3.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_3_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [20]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[model_3_checkpoint_callback, tensor_call], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(checkdir)
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 3s 142ms/step - loss: 0.9001 - auc: 0.4009 - val_loss: 0.8288 - val_auc: 0.4074
Epoch 2/100
18/18 [==============================] - 1s 28ms/step - loss: 0.8500 - auc: 0.4360 - val_loss: 0.7840 - val_auc: 0.4439
Epoch 3/100
18/18 [==============================] - 1s 30ms/step - loss: 0.8066 - auc: 0.4727 - val_loss: 0.7453 - val_auc: 0.4738
Epoch 4/100
18/18 [==============================] - 0s 24ms/step - loss: 0.7690 - auc: 0.5080 - val_loss: 0.7129 - val_auc: 0.5053
Epoch 5/100
18/18 [==============================] - 1s 28ms/step - loss: 0.7372 - auc: 0.5413 - val_loss: 0.6833 - val_auc: 0.5510
Epoch 6/100
18/18 [==============================] - 0s 27ms/step - loss: 0.7084 - auc: 0.5772 - val_loss: 0.6591 - val_auc: 0.5901
Epoch 7/100
18/18 [==============================] - 1s 28ms/step - loss: 0.6845 - auc: 0.6083 - val_loss: 0.6384 - val_auc: 0.6255
Epoch 8/100
18/18 [==============================] - 0s 26ms/step - loss: 0

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.835058,0.867036,0.564767,0.694268,0.788413
1,Validacion,0.834559,0.890625,0.500000,0.708333,0.770270


### Early Stopping
Los hyperparámetrso principales de este callback son:
- monitor: indica la variable a monitorear
- min_delta: la minima diferencia que se considera como mejora
- patience: cantidad de epochs sin mejoras antes de parar

In [21]:
stop_early = EarlyStopping(monitor='val_auc', patience=20)
name = 'early_stop'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
model_4_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [22]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=50, validation_data=(x_valid_norm, y_valid), callbacks=[model_4_checkpoint_callback, stop_early, tensor_call], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(checkdir)
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/50
18/18 [==============================] - 2s 87ms/step - loss: 0.4774 - auc: 0.8347 - val_loss: 0.4716 - val_auc: 0.8359
Epoch 2/50
18/18 [==============================] - 0s 12ms/step - loss: 0.4773 - auc: 0.8349 - val_loss: 0.4717 - val_auc: 0.8355
Epoch 3/50
18/18 [==============================] - 0s 12ms/step - loss: 0.4772 - auc: 0.8350 - val_loss: 0.4718 - val_auc: 0.8350
Epoch 4/50
18/18 [==============================] - 0s 14ms/step - loss: 0.4771 - auc: 0.8349 - val_loss: 0.4719 - val_auc: 0.8359
Epoch 5/50
18/18 [==============================] - 0s 14ms/step - loss: 0.4770 - auc: 0.8346 - val_loss: 0.4719 - val_auc: 0.8352
Epoch 6/50
18/18 [==============================] - 0s 9ms/step - loss: 0.4769 - auc: 0.8347 - val_loss: 0.4720 - val_auc: 0.8359
Epoch 7/50
18/18 [==============================] - 0s 11ms/step - loss: 0.4768 - auc: 0.8351 - val_loss: 0.4720 - val_auc: 0.8357
Epoch 8/50
18/18 [==============================] - 0s 11ms/step - loss: 0.4767 - au

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.835044,0.864266,0.564767,0.689873,0.787879
1,Validacion,0.834099,0.890625,0.500000,0.708333,0.770270


Se puede evidenciar como al utilizar early stopping el entrenamiento finaliza antes debido a que las mejoras son reducidas

## Ajuste de Hyperparámetros

### Learning rate
Por defecto en SGD el learning rate que utiliza keras es 0.01

In [23]:
learning_rates = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
best = 0
best_name = ''
for i in learning_rates:
    name = 'learning_{}'.format(i)
    checkdir = join(checkpoints_path, name)
    tensordir = join(tensor_path, name)
    tensor_call = TensorBoard(log_dir=tensordir)
    learning_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)


    learning = Sequential()
    learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    learning.compile(optimizer=SGD(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[learning_checkpoint_callback, stop_early, tensor_call], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    learning.load_weights(checkdir)
    metrics = verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    if best < auc_res[i][0]:
        best_name = checkdir
        best = auc_res[i][0]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.464154,0.582721,0.318934,0.74954,0.848805,0.830423,0.83318,0.838235,0.839614,0.839614,0.840074,0.841912,0.84329,0.84421,0.836397,0.845588,0.838695
1,Epochs,37.000000,20.000000,20.000000,20.00000,44.000000,20.000000,20.00000,20.000000,30.000000,27.000000,54.000000,49.000000,48.00000,20.00000,40.000000,50.000000,21.000000


Es interesante observar como para los learning rate mas chicos, el entrenamiento se corta debido al early stoping en los primeros 15 epochs, ya que dentro de esos primeros 15 las mejoras son pocas debido a su avance "suave", en comparación con los learning rates mas altos que obtienen una mejor optimización en mayor cantidad de epochs

In [24]:
# Cargo el mejor modelo entrenado
learning.load_weights(best_name)
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.820275,0.814404,0.652850,0.652850,0.814404
1,Validacion,0.848805,0.859375,0.647059,0.709677,0.820896


#### Scheduling learning rate

In [25]:
learning_schedule = ExponentialDecay(initial_learning_rate=2.7, decay_steps=100, decay_rate=0.96)
name = 'exponetial'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
    
exponential_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)


In [26]:
learning = Sequential()
learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
learning.compile(optimizer=SGD(learning_rate=learning_schedule), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [27]:
history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[exponential_checkpoint_callback, stop_early, tensor_call], 
                      verbose=1)
# Cargo el mejor modelo entrenado
learning.load_weights(checkdir)
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 2s 139ms/step - loss: 0.5832 - auc: 0.7541 - val_loss: 0.5762 - val_auc: 0.7431
Epoch 2/100
18/18 [==============================] - 0s 10ms/step - loss: 0.5450 - auc: 0.7988 - val_loss: 0.6890 - val_auc: 0.7027
Epoch 3/100
18/18 [==============================] - 1s 29ms/step - loss: 0.5361 - auc: 0.8010 - val_loss: 0.5314 - val_auc: 0.8157
Epoch 4/100
18/18 [==============================] - 1s 29ms/step - loss: 0.5148 - auc: 0.8115 - val_loss: 0.5166 - val_auc: 0.8332
Epoch 5/100
18/18 [==============================] - 0s 13ms/step - loss: 0.5404 - auc: 0.8009 - val_loss: 0.5092 - val_auc: 0.8120
Epoch 6/100
18/18 [==============================] - 0s 13ms/step - loss: 0.5245 - auc: 0.8027 - val_loss: 0.5331 - val_auc: 0.7955
Epoch 7/100
18/18 [==============================] - 0s 12ms/step - loss: 0.5183 - auc: 0.8108 - val_loss: 0.5343 - val_auc: 0.8217
Epoch 8/100
18/18 [==============================] - 0s 8ms/step - loss: 0.

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.810156,0.792244,0.632124,0.619289,0.80112
1,Validacion,0.846507,0.843750,0.705882,0.705882,0.84375


### Momentum

In [28]:
momentum_arr = [0.5, 0.6, 0.85, 0.9, 0.95, 0.99, 0.999]

arr_of_metrics = []
auc_res = {'momentum':['AUC', 'Epochs']}
for i in momentum_arr:
    name = 'momentum_{}'.format(i)
    checkdir = join(checkpoints_path, name)
    tensordir = join(tensor_path, name)
    tensor_call = TensorBoard(log_dir=tensordir)

    momentum_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    
    mom_model = Sequential()
    mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback, tensor_call], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    mom_model.load_weights(checkdir)
    metrics = verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,momentum,0.5,0.6,0.85,0.9,0.95,0.99,0.999
0,AUC,0.847426,0.844669,0.839614,0.845588,0.832721,0.842831,0.827206
1,Epochs,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [29]:
name = 'momentum'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
momentum_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

mom_model = Sequential()
mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=0.6), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback, stop_early, tensor_call], 
                          verbose=1)
# Cargo el mejor modelo entrenado
mom_model.load_weights(checkdir)
verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 2s 114ms/step - loss: 0.6739 - auc: 0.7595 - val_loss: 0.6068 - val_auc: 0.7619
Epoch 2/100
18/18 [==============================] - 1s 34ms/step - loss: 0.5841 - auc: 0.7788 - val_loss: 0.6337 - val_auc: 0.7677
Epoch 3/100
18/18 [==============================] - 0s 13ms/step - loss: 0.5860 - auc: 0.7750 - val_loss: 0.6248 - val_auc: 0.7569
Epoch 4/100
18/18 [==============================] - 0s 12ms/step - loss: 0.6032 - auc: 0.7609 - val_loss: 0.7277 - val_auc: 0.7574
Epoch 5/100
18/18 [==============================] - 1s 31ms/step - loss: 0.6286 - auc: 0.7663 - val_loss: 0.5928 - val_auc: 0.7852
Epoch 6/100
18/18 [==============================] - 0s 13ms/step - loss: 0.5567 - auc: 0.7966 - val_loss: 0.5995 - val_auc: 0.7397
Epoch 7/100
18/18 [==============================] - 1s 32ms/step - loss: 0.5787 - auc: 0.7766 - val_loss: 0.5651 - val_auc: 0.8350
Epoch 8/100
18/18 [==============================] - 0s 11ms/step - loss: 0

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.817806,0.770083,0.678756,0.612150,0.817647
1,Validacion,0.844669,0.812500,0.764706,0.684211,0.866667


### Optimizador Adam

In [30]:
%%time
adam_lr = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
for i in adam_lr:
    name = 'Adam_{}'.format(i)
    checkdir = join(checkpoints_path, name)
    tensordir = join(tensor_path, name)
    tensor_call = TensorBoard(log_dir=tensordir)
    adam_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    adam_mod = Sequential()
    adam_mod.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    adam_mod.compile(optimizer=Adam(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = adam_mod.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback, tensor_call], 
                          verbose=1, use_multiprocessing=True)
    # Cargo el mejor modelo entrenado
    adam_mod.load_weights(checkdir)
    metrics = verify_model(adam_mod, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

Wall time: 7min 55s


,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.782629,0.605239,0.821232,0.832261,0.836857,0.841912,0.845588,0.846507,0.846507,0.846048,0.846967,0.84375,0.840993,0.837776,0.850184,0.841912,0.835018
1,Epochs,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.00000,99.000000,99.000000,99.000000,99.000000,99.000000


In [31]:
%%time
name = 'Adam'
checkdir = join(checkpoints_path, name)
tensordir = join(tensor_path, name)
tensor_call = TensorBoard(log_dir=tensordir)
adam_checkpoint_callback = ModelCheckpoint(filepath=checkdir,
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

adam_model = Sequential()
adam_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
adam_model.compile(optimizer=Adam(learning_rate=4.7), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = adam_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback, tensor_call], 
                          verbose=1)
# Cargo el mejor modelo entrenado
adam_model.load_weights(checkdir)
verify_model(adam_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 4s 210ms/step - loss: 3.4225 - auc: 0.6856 - val_loss: 1.4663 - val_auc: 0.6935
Epoch 2/100
18/18 [==============================] - 0s 11ms/step - loss: 1.3603 - auc: 0.7026 - val_loss: 1.1140 - val_auc: 0.6305
Epoch 3/100
18/18 [==============================] - 0s 24ms/step - loss: 1.6656 - auc: 0.6990 - val_loss: 1.6291 - val_auc: 0.7486
Epoch 4/100
18/18 [==============================] - 0s 12ms/step - loss: 1.3014 - auc: 0.7455 - val_loss: 1.5431 - val_auc: 0.7479
Epoch 5/100
18/18 [==============================] - 0s 17ms/step - loss: 2.2450 - auc: 0.7000 - val_loss: 0.9774 - val_auc: 0.7477
Epoch 6/100
18/18 [==============================] - 0s 27ms/step - loss: 1.4598 - auc: 0.7056 - val_loss: 0.9155 - val_auc: 0.7900
Epoch 7/100
18/18 [==============================] - 0s 20ms/step - loss: 1.4039 - auc: 0.7300 - val_loss: 1.0146 - val_auc: 0.8100
Epoch 8/100
18/18 [==============================] - 0s 10ms/step - loss: 1

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.806740,0.831025,0.606218,0.657303,0.797872
1,Validacion,0.831801,0.843750,0.558824,0.655172,0.782609


Evaluación contra test

In [32]:
print('Utilizando SGD')
verify_model(mom_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando SGD


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.817806,0.770083,0.678756,0.612150,0.817647
1,Test,0.853333,0.773333,0.707317,0.630435,0.828571


In [33]:
print('Utilizando Adam')
verify_model(adam_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando Adam


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.806740,0.831025,0.606218,0.657303,0.797872
1,Test,0.841301,0.853333,0.707317,0.725000,0.842105


Finalmente se obtiene que el modelo que utiliza como optimizador SGD y los hyperparámetros optimizados fue el mejor de los entrenados hasta ahora

Cabe destacar una particularidad, que los resustados en Test son mejores que en el set de validacion, e incluso mejores que el set de train, con lo cual concluir que la metrica de nuestro modelo es AUC=0.86 no sería correcto, ya que es el caso en el que tenemos un estimador con alta varianza, por lo tanto una estimación utilizando k-folding sería mas adecuada (si hay tiempo la hago...)

In [34]:
f2_threshold(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)

TypeError: fbeta_score() missing 1 required keyword-only argument: 'beta'